# laden

In [4]:
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
import cv2

import random
from tqdm import tqdm

import re
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger

from keras.applications import MobileNetV2

from keras.layers import *
from keras.models import Model

Using TensorFlow backend.


In [5]:
def openTar(Name):
  tar = tarfile.open(Name)
  tar.extractall()
  tar.close()
  print(Name + " entpacked")


In [6]:
!curl https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/labels.tar.gz --output "labels.tar.gz"
openTar("labels.tar.gz")
!rm labels.tar.gz

StringToJson = 'publaynet/val.json'
StringToFolder = 'publaynet/val/'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  313M  100  313M    0     0   110M      0  0:00:02  0:00:02 --:--:--  110M
labels.tar.gz entpacked


In [32]:
!curl https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/examples.tar.gz --output "examples.tar.gz"
openTar("examples.tar.gz")
!rm examples.tar.gz

StringToJson = 'examples/samples.json'
StringToFolder = 'examples/'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6309k  100 6309k    0     0  9603k      0 --:--:-- --:--:-- --:--:-- 9589k
examples.tar.gz entpacked


In [33]:
with open(StringToJson, 'r') as fp:
    samples = json.load(fp)
# Index images
images = {}
for image in samples['images']:
    images[image['id']] = {'file_name': image['file_name'], 'annotations': []}
for ann in samples['annotations']:
    images[ann['image_id']]['annotations'].append(ann)


# Umwandlung

In [28]:
def markup(BildWeg, annotations):
  # laden der Zusatdaten
  Object = {}

  Object['image'] = BildWeg
  Object['annotations'] = []

  for annotation in annotations:
    # "bbox": [37.59, 433.64, 251.07, 20.38]
    temp = {}
    temp['label'] = 'found'

    # Left, Top, Right, Bottom = annotation['bbox'][0], annotation['bbox'][1], annotation['bbox'][2], annotation['bbox'][3]

    temp_coordinates = {}
    temp_coordinates['x'], temp_coordinates['y'], temp_coordinates['width'], temp_coordinates['height'] = annotation['bbox'][0], annotation['bbox'][1], annotation['bbox'][2], annotation['bbox'][3]


    temp['coordinates'] = temp_coordinates

    Object['annotations'].append(temp)

  return Object

In [34]:
Ziel = []

for i, (_, image) in enumerate(images.items()):
  temp = markup(image['file_name'], image['annotations'])
  Ziel.append(temp)

In [35]:
print(Ziel[0])

{'image': 'PMC5491943_00004.jpg', 'annotations': [{'label': 'found', 'coordinates': {'x': 121.89, 'y': 41.8, 'width': 427.99, 'height': 34.5}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 83.47, 'width': 427.94, 'height': 46.5}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 163.31, 'width': 428.08, 'height': 83.69}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 254.37, 'width': 427.96, 'height': 108.09}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 369.82, 'width': 214.62, 'height': 10.51}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 653.61, 'width': 340.22, 'height': 10.51}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 671.48, 'width': 427.98, 'height': 59.3}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 387.69, 'width': 428.0, 'height': 221.88}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 144.1, 'width': 104.54, 'height': 12.47}}, {'label': 'found', 'coordinates': {'x': 121.89, 'y': 632.39, 'width': 61.4, 'height

In [36]:
with open('Ziel.json', 'w') as json_file:
  json.dump(Ziel, json_file)